In [2]:
!pwd 
!source  .venv/bin/activate

/home/ramma/XPROJECT/deep-learning


In [11]:
import torch 
from torchvision import datasets 

data_folder   = 'data' 

fmnist_train  = datasets.FashionMNIST(data_folder, download=True, train=True  )
fmnist_val    = datasets.FashionMNIST(data_folder, download=True, train=False )

tr_images     = fmnist_train.data 
tr_targets    = fmnist_train.targets

val_images    = fmnist_val.data 
val_targets   = fmnist_val.targets 





In [13]:
import matplotlib.pyplot as plt 
import numpy as np
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Without BatchNormalization

In [14]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y):
        x  = x.float()/(255*10000)
        x  = x.view(-1, 28*28)
        self.x, self.y = x, y
    def __getitem__(self, ix): 
        x, y = self.x[ix], self.y[ix]
        return x.to(device), y.to(device)
    def __len__(self):
        return len(self.x)

def get_data():     
    train = FMNISTDataset(tr_images, tr_targets)     
    trn_dl = DataLoader(train, batch_size=32, shuffle=True)
    val = FMNISTDataset(val_images, val_targets)     
    val_dl = DataLoader(val, batch_size=len(val_images), shuffle=True)
    return trn_dl, val_dl

trn_dl, val_dl = get_data()

In [ ]:
from torch.optim import SGD, Adam

def get_model():
    class neuralnet(nn.Module):
        def __init__(self):
            super().__init__()
            self.input_to_hidden_layer = nn.Linear(784,1000)
            self.hidden_layer_activation = nn.ReLU()
            self.hidden_to_output_layer = nn.Linear(1000,10)
        def forward(self, x):
            x = self.input_to_hidden_layer(x)
            x1 = self.hidden_layer_activation(x)
            x2= self.hidden_to_output_layer(x1)
            return x2, x1
    model = neuralnet().to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

def train_batch(x, y, model, opt, loss_fn):
    prediction = model(x)[0]
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

@torch.no_grad()
def val_loss(x, y, model):
    model.eval()
    prediction = model(x)[0]
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

def accuracy(x, y, model):
    with torch.no_grad():
        prediction = model(x)[0]
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

model, loss_fn, optimizer = get_model()